In [1]:
# Optional: install if missing (uncomment to run)
import sys
!{sys.executable} -m pip install MetaTrader5 stable-baselines3[extra] gymnasium numpy pandas matplotlib


  Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (19 kB)
  Using cached pygame-2.6.1-cp313-cp313-win_amd64.whl.metadata (13 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached rich-14.2.0-py3-none-any.whl.metadata (18 kB)
  Using cached Shimmy-1.3.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached AutoROM-0.6.1-py3-none-any.whl.metadata (2.4 kB)
  Using cached AutoROM.accept-rom-license-0.6.1.tar.gz (434 kB)
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
INFO: pip is looking at multiple versions of shimmy[atari] to determine which version is compatible with other requirements. This could take a while.
  Usin

In [12]:
# Cell 2
import os, glob, time, json
from datetime import datetime
import numpy as np
import pandas as pd
import MetaTrader5 as mt5
import matplotlib.pyplot as plt

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecNormalize

# === PATHS (from your config) ===
DATA_DIR = os.path.join("data", "multiasset")
MODEL_DIR = os.path.join("models", "multiasset")
EMBED_FILE = os.path.join(MODEL_DIR, "asset_embeddings.npy")
ASSET_MAP_FILE = os.path.join(DATA_DIR, "asset_to_idx.csv")

MODEL_FILE = os.path.join(MODEL_DIR, "ppo_multiasset.zip")
VEC_FILE = os.path.join(MODEL_DIR, "vec_normalize.pkl")
METRICS_FILE = os.path.join(MODEL_DIR, "eval_metrics.json")

# Runtime config
WINDOW = 50
TF_MT5 = mt5.TIMEFRAME_M1   # M1 as used earlier
DRY_RUN = False              # KEEP True while testing
DEFAULT_LOT = 0.1           # as requested
MAX_POS_PER_SYMBOL = 2

# SL/TP settings: SL computed from volatility (function below); TP = 3 * SL
DEFAULT_SL_PIPS_FALLBACK = 20
TP_MULT = 3

# Logging
LOG_DIR = os.path.join(MODEL_DIR, "live_logs")
os.makedirs(LOG_DIR, exist_ok=True)
LOG_FILE = os.path.join(LOG_DIR, "live_trade_logs.csv")


In [24]:
# Cell 3
def make_safe_name(sym: str) -> str:
    return sym.replace(" ", "_").replace("/", "_").replace("(", "").replace(")", "").replace(".", "_")

def raw_from_safe(safe: str) -> str:
    return safe.replace("_", " ")

def safe_from_raw(raw: str) -> str:
    return make_safe_name(raw)


In [25]:
# Cell 4
# Load per-asset scalers saved as CSVs: {safe}_scaler.csv with columns mean/std
def load_scalers_from_csv(data_dir=DATA_DIR):
    scalers = {}
    for p in sorted(glob.glob(os.path.join(data_dir, "*_scaler.csv"))):
        safe = os.path.basename(p).replace("_scaler.csv","")
        df = pd.read_csv(p, index_col=0)
        # ensure ordering of mean/std columns matches our feature order: o_pc,h_pc,l_pc,c_pc,v_pc
        scalers[safe] = {"mean": df["mean"], "std": df["std"]}
    return scalers

def load_datasets(data_dir=DATA_DIR, window=WINDOW):
    datasets = {}
    for p in sorted(glob.glob(os.path.join(data_dir, "*_normalized.csv"))):
        safe = os.path.basename(p).replace("_normalized.csv","")
        df = pd.read_csv(p, index_col=0, parse_dates=True)
        # ensure expected columns exist; otherwise try conversion as earlier notebooks did
        expected = ['o_pc','h_pc','l_pc','c_pc','v_pc','Close_raw']
        if all(c in df.columns for c in expected):
            df = df[expected].dropna()
        else:
            # attempt auto-convert if original OHLCV present
            if all(c in df.columns for c in ['open','high','low','close','volume']):
                tmp = pd.DataFrame(index=df.index)
                tmp['o_pc'] = df['open'].pct_change()
                tmp['h_pc'] = df['high'].pct_change()
                tmp['l_pc'] = df['low'].pct_change()
                tmp['c_pc'] = df['close'].pct_change()
                tmp['v_pc'] = df['volume'].pct_change()
                tmp['Close_raw'] = df['close']
                df = tmp.dropna()
            else:
                raise ValueError(f"{p} missing expected columns and cannot convert")
        if len(df) > window:
            datasets[safe] = df
    return datasets

def load_embeddings(embed_file=EMBED_FILE, data_dir=DATA_DIR):
    if not os.path.exists(embed_file):
        print("No embeddings file found at", embed_file)
        return {}
    emb = np.load(embed_file, allow_pickle=True)
    # emb likely is ndarray shape (n_assets, embed_dim)
    # Map using asset_to_idx.csv if present
    emb_dict = {}
    if os.path.exists(ASSET_MAP_FILE):
        am = pd.read_csv(ASSET_MAP_FILE, index_col=0, header=None).iloc[:,0].to_dict()
        # am is mapping safe->idx
        for safe, idx in am.items():
            idx = int(idx)
            if idx < emb.shape[0]:
                emb_dict[safe] = np.array(emb[idx], dtype=np.float32)
    else:
        # fallback: map by order of normalized CSVs if sizes match
        csvs = sorted(glob.glob(os.path.join(data_dir, "*_normalized.csv")))
        safe_list = [os.path.basename(p).replace("_normalized.csv","") for p in csvs]
        if len(safe_list) == emb.shape[0]:
            for i, safe in enumerate(safe_list):
                emb_dict[safe] = np.array(emb[i], dtype=np.float32)
        else:
            print("Warning: embedding count and CSV count mismatch; manual mapping required.")
    return emb_dict

# load
scalers = load_scalers_from_csv(DATA_DIR)
datasets = load_datasets(DATA_DIR, WINDOW)
embeddings = load_embeddings(EMBED_FILE, DATA_DIR)

print("Loaded assets:", len(datasets))
print("Loaded scalers:", len(scalers))
print("Loaded embeddings:", len(embeddings))


Loaded assets: 16
Loaded scalers: 16
Loaded embeddings: 17


In [26]:
# Cell 5
# Load PPO model
if not os.path.exists(MODEL_FILE):
    raise FileNotFoundError("Model not found: " + MODEL_FILE)
model = PPO.load(MODEL_FILE)
print("Loaded PPO from", MODEL_FILE)

# Try to load VecNormalize (optional)
vecnorm = None
if os.path.exists(VEC_FILE):
    try:
        vecnorm = VecNormalize.load(VEC_FILE)
        print("Loaded VecNormalize from", VEC_FILE)
    except Exception as e:
        print("VecNormalize load failed:", e)
        vecnorm = None


Loaded PPO from models\multiasset\ppo_multiasset.zip
VecNormalize load failed: VecNormalize.load() missing 1 required positional argument: 'venv'


In [27]:
# Cell 6
def fetch_and_build_obs(raw_symbol, window, scalers, embeddings, datasets, safe_list):
    """
    Build observation matching training:
    obs shape = (window, n_features=5 + 1 + embed_dim)
      -> 5: o_pc,h_pc,l_pc,c_pc,v_pc
      -> 1: volatility column (c_pc) per timestep (training used balance column previously; we use c_pc as volatility column)
         NOTE: original training used 'balance' column as +1; earlier env used balance=1.0. To be exact, we will include 'balance' column instead of vol column so obs_dim matches training:
         training obs was [OHLCV_pctnorm, balance, embedding]. So we'll include balance (1.0) and embedding; volatility used for SL only.
    """
    safe = safe_from_raw(raw_symbol)
    if safe not in datasets:
        print("No dataset for", safe)
        return None, None, None
    df = datasets[safe]
    if len(df) < window:
        print("Not enough data for", safe)
        return None, None, None

    window_df = df.iloc[-window:]
    # features (pct-changes)
    try:
        features = window_df[['o_pc','h_pc','l_pc','c_pc','v_pc']].values.astype(np.float32)  # shape (window,5)
        last_price = float(window_df['Close_raw'].iloc[-1])
    except KeyError as e:
        print("Column missing:", e)
        return None, None, None

    # scale using scalers dict (mean/std may be pandas Series)
    if safe not in scalers:
        print("No scaler for", safe)
        return None, None, None
    s = scalers[safe]
    # accept either pandas Series or numpy arrays
    mean = np.array(s['mean'].values if hasattr(s['mean'], "values") else s['mean'], dtype=np.float32)
    std  = np.array(s['std'].values if hasattr(s['std'], "values") else s['std'], dtype=np.float32)
    std = np.where(std == 0, 1e-8, std)
    features_scaled = (features - mean) / std

    # embedding vector
    if safe in embeddings:
        emb_vec = np.array(embeddings[safe], dtype=np.float32)
    else:
        # fallback zero vector of expected embedding dim (use first embedding length if possible)
        if len(embeddings)>0:
            emb_dim = next(iter(embeddings.values())).shape[0]
            emb_vec = np.zeros(emb_dim, dtype=np.float32)
        else:
            emb_vec = np.zeros(8, dtype=np.float32)  # fallback to 8

    emb_block = np.repeat(emb_vec.reshape(1, -1), window, axis=0)  # (window, embed_dim)

    # Balance column (training used balance; set to 1.0)
    balance_col = np.full((window,1), 1.0, dtype=np.float32)

    # Final obs: [features_scaled (5) , balance (1) , embedding (embed_dim)]
    obs = np.concatenate([features_scaled, balance_col, emb_block], axis=1).astype(np.float32)  # (window, 5+1+embed_dim)

    # VecNormalize optional normalization: apply if present (flatten-first)
    if vecnorm is not None:
        try:
            # vecnorm.obs_rms.mean shape matches flattened obs; we need to flatten timestep dimension
            flat = obs.reshape(1, -1)  # shape (1, window * feat)
            mean_rms = getattr(vecnorm.obs_rms, "mean", None)
            var_rms = getattr(vecnorm.obs_rms, "var", None)
            if mean_rms is not None and var_rms is not None and len(mean_rms) == flat.shape[1]:
                flat_norm = (flat - mean_rms) / np.sqrt(var_rms + 1e-8)
                obs = flat_norm.reshape(obs.shape).astype(np.float32)
            else:
                # fallback: use vecnorm.normalize_obs if available (some versions)
                try:
                    obs = vecnorm.normalize_obs(obs, False)
                except Exception:
                    pass
        except Exception as e:
            print("VecNormalize apply failed:", e)

    # estimate vol for lot/SL computation (std of c_pc)
    vol_est = float(window_df['c_pc'].std())

    return obs, vol_est, last_price


In [28]:
# Cell 7
def estimate_sl_pips_from_vol(raw_symbol, last_price, vol_est, min_pips=5, max_pips=200):
    """
    Heuristic placeholder for 'learned' SL:
    - vol_est is std of c_pc (pct-change) per bar
    - approximate absolute move = vol_est * last_price
    - convert to pips: pip = 0.0001 (or 0.01 for JPY)
    - produce sl_pips clipped to [min_pips, max_pips]
    """
    if "JPY" in raw_symbol.upper():
        pip = 0.01
    else:
        pip = 0.0001
    # absolute expected move:
    abs_move = vol_est * last_price
    if abs_move <= 0:
        sl_pips = DEFAULT_SL_PIPS_FALLBACK
    else:
        sl_raw = abs_move / pip
        # safety scale factor
        sl_pips = float(np.clip(np.round(sl_raw * 1.5), min_pips, max_pips))
    return int(sl_pips)

def compute_lot_fixed(default_lot=DEFAULT_LOT):
    return float(default_lot)


In [29]:
# Cell 8
def get_positions_for_symbol(symbol):
    pos = mt5.positions_get(symbol=symbol)
    return [] if pos is None else list(pos)

def place_market_order(symbol, direction, lot, sl_price=None, tp_price=None):
    """
    direction: "BUY" or "SELL"
    returns dict-like result (simulated if DRY_RUN)
    """
    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        print("❌ Cannot get tick for", symbol)
        return None
    price = float(tick.ask if direction=="BUY" else tick.bid)
    if DRY_RUN:
        # simulated response
        return {"retcode": 10009, "price": price, "comment":"DRY_RUN", "direction":direction}
    req = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": float(lot),
        "type": mt5.ORDER_TYPE_BUY if direction=="BUY" else mt5.ORDER_TYPE_SELL,
        "price": price,
        "sl": float(sl_price) if sl_price is not None else 0.0,
        "tp": float(tp_price) if tp_price is not None else 0.0,
        "deviation": 20,
        "magic": 234000,
        "comment": "ppo_multiasset_live",
        "type_filling": mt5.ORDER_FILLING_FOK,
    }
    res = mt5.order_send(req)
    return res

def close_position_by_ticket(ticket):
    pos_list = mt5.positions_get(ticket=ticket)
    if not pos_list:
        return None
    p = pos_list[0]
    symbol = p.symbol
    if getattr(p, "type", None) == 0:  # BUY => close with SELL
        order_type = mt5.ORDER_TYPE_SELL
        price = mt5.symbol_info_tick(symbol).bid
    else:
        order_type = mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(symbol).ask
    if DRY_RUN:
        return {"retcode":10009, "comment":"DRY_RUN_CLOSE", "ticket": ticket}
    req = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": float(p.volume),
        "type": order_type,
        "position": int(ticket),
        "price": price,
        "deviation": 20,
        "magic": 234000,
        "comment": "auto_close",
    }
    return mt5.order_send(req)


In [30]:
# Cell 9
def run_once_predict_and_manage(model, safe_list, scalers, embeddings, datasets):
    header = not os.path.exists(LOG_FILE)
    acct = mt5.account_info()
    balance = float(acct.balance) if acct else 10000.0

    # iterate assets
    for safe in safe_list:
        raw = raw_from_safe(safe)   # MT5 symbol name guess
        print("\n---", raw, "(", safe, ") ---")
        obs, vol, last_price = fetch_and_build_obs(raw, WINDOW, scalers, embeddings, datasets, safe_list)
        if obs is None:
            print("Skip", raw)
            continue

        # model expects batch dim
        try:
            action, _ = model.predict(obs[np.newaxis, ...], deterministic=True)
            a = int(action[0]) if isinstance(action, (list,tuple,np.ndarray)) else int(action)
        except Exception as e:
            print("Prediction error:", e)
            continue

        print("Signal:", a, " (0=HOLD,1=BUY,2=SELL)")

        # get current positions for symbol
        positions = get_positions_for_symbol(raw)
        print("Existing positions:", len(positions))

        # Close opposing positions if reversal
        if a == 1:
            # BUY signal -> close SELL positions (type==1)
            for p in positions:
                if getattr(p,"type",None) == 1:
                    print("Closing opposing SELL ticket", p.ticket)
                    close_position_by_ticket(p.ticket)
        elif a == 2:
            for p in positions:
                if getattr(p,"type",None) == 0:
                    print("Closing opposing BUY ticket", p.ticket)
                    close_position_by_ticket(p.ticket)

        # refresh positions
        positions = get_positions_for_symbol(raw)
        if len(positions) >= MAX_POS_PER_SYMBOL:
            print("Max positions reached for", raw)
        else:
            if a == 0:
                print("HOLD — no trade")
            else:
                direction = "BUY" if a==1 else "SELL"
                lot = compute_lot_fixed(DEFAULT_LOT)
                # estimate SL pips from volatility, then compute TP = 3 * SL
                sl_pips = estimate_sl_pips_from_vol(raw, last_price, vol)
                tp_pips = int(sl_pips * TP_MULT)
                # convert pips to price levels
                if "JPY" in raw.upper():
                    pip = 0.01
                else:
                    pip = 0.0001
                if direction == "BUY":
                    sl_price = last_price - sl_pips * pip
                    tp_price = last_price + tp_pips * pip
                else:
                    sl_price = last_price + sl_pips * pip
                    tp_price = last_price - tp_pips * pip

                res = place_market_order(raw, direction, lot, sl_price, tp_price)
                # normalize response
                if isinstance(res, dict):
                    retcode = res.get("retcode")
                    comment = res.get("comment")
                    exec_price = res.get("price", last_price)
                else:
                    retcode = getattr(res, "retcode", None)
                    comment = getattr(res, "comment", "")
                    exec_price = last_price

                entry = {
                    "timestamp": datetime.utcnow().isoformat(),
                    "safe": safe,
                    "symbol": raw,
                    "action": direction,
                    "lot": lot,
                    "exec_price": exec_price,
                    "sl_price": sl_price,
                    "tp_price": tp_price,
                    "sl_pips": sl_pips,
                    "tp_pips": tp_pips,
                    "retcode": retcode,
                    "comment": comment,
                    "dry_run": DRY_RUN
                }
                pd.DataFrame([entry]).to_csv(LOG_FILE, mode="a", index=False, header=header)
                header = False
                print("Placed", direction, "lot", lot, "retcode", retcode)

        # Trailing: update SL for existing positions if profitable (simple rule)
        for p in get_positions_for_symbol(raw):
            # compute trailing SL target in price; use trailing pips = max(5, sl_pips//2) as example
            try:
                trail_pips = max(5, int(estimate_sl_pips_from_vol(raw, last_price, vol) // 2))
            except Exception:
                trail_pips = 5
            if "JPY" in raw.upper():
                pip = 0.01
            else:
                pip = 0.0001
            if getattr(p,"type",None) == 0:  # BUY
                # new SL = current price - trail_pips
                cur = mt5.symbol_info_tick(raw).bid
                new_sl = float(cur - trail_pips * pip)
                if DRY_RUN:
                    print(f"[DRY] Would set trailing SL for ticket {p.ticket} -> {new_sl}")
                else:
                    req = {"action": mt5.TRADE_ACTION_SLTP, "symbol": raw, "position": int(p.ticket), "sl": new_sl, "tp": float(p.tp) if getattr(p,"tp",None) else 0.0}
                    r = mt5.order_send(req)
                    print("Modify SL result:", getattr(r,"retcode",None))
            else:  # SELL
                cur = mt5.symbol_info_tick(raw).ask
                new_sl = float(cur + trail_pips * pip)
                if DRY_RUN:
                    print(f"[DRY] Would set trailing SL for ticket {p.ticket} -> {new_sl}")
                else:
                    req = {"action": mt5.TRADE_ACTION_SLTP, "symbol": raw, "position": int(p.ticket), "sl": new_sl, "tp": float(p.tp) if getattr(p,"tp",None) else 0.0}
                    r = mt5.order_send(req)
                    print("Modify SL result:", getattr(r,"retcode",None))

    print("\nSingle pass complete.")


In [23]:
# Cell 10
safe_list = sorted(datasets.keys())
print("Assets to process:", safe_list)
run_once_predict_and_manage(model, safe_list, scalers, embeddings, datasets)


Assets to process: ['EURUSD', 'Jump_100_Index', 'Jump_10_Index', 'Jump_25_Index', 'Jump_50_Index', 'Jump_75_Index', 'Volatility_100_1s_Index', 'Volatility_100_Index', 'Volatility_10_1s_Index', 'Volatility_10_Index', 'Volatility_25_1s_Index', 'Volatility_25_Index', 'Volatility_50_1s_Index', 'Volatility_50_Index', 'Volatility_75_1s_Index', 'Volatility_75_Index']

--- EURUSD ( EURUSD ) ---
Signal: 2  (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
❌ Cannot get tick for EURUSD
Placed SELL lot 0.1 retcode None

--- Jump 100 Index ( Jump_100_Index ) ---
Signal: 1  (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
❌ Cannot get tick for Jump 100 Index
Placed BUY lot 0.1 retcode None

--- Jump 10 Index ( Jump_10_Index ) ---
Signal: 1  (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
❌ Cannot get tick for Jump 10 Index
Placed BUY lot 0.1 retcode None

--- Jump 25 Index ( Jump_25_Index ) ---
Signal: 0  (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
HOLD — no trade

--- Jump 50 Index ( Jump_50_Index ) ---
Signal: 1

C:\Users\Slick\AppData\Local\Temp\ipykernel_11712\2806280530.py:80: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
C:\Users\Slick\AppData\Local\Temp\ipykernel_11712\2806280530.py:80: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
C:\Users\Slick\AppData\Local\Temp\ipykernel_11712\2806280530.py:80: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
C:\Users\Slick\AppData\Local\Temp\ipykernel_11712

Signal: 1  (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
❌ Cannot get tick for Volatility 50 Index
Placed BUY lot 0.1 retcode None

--- Volatility 75 1s Index ( Volatility_75_1s_Index ) ---
Signal: 1  (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
❌ Cannot get tick for Volatility 75 1s Index
Placed BUY lot 0.1 retcode None

--- Volatility 75 Index ( Volatility_75_Index ) ---
Signal: 0  (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
HOLD — no trade

Single pass complete.


C:\Users\Slick\AppData\Local\Temp\ipykernel_11712\2806280530.py:80: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
C:\Users\Slick\AppData\Local\Temp\ipykernel_11712\2806280530.py:80: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),


In [ ]:
# Cell 11 (optional)
def simulate_trades_on_history(datasets, model, assets=None, window=WINDOW, horizon=10):
    trades = []
    assets = assets or list(datasets.keys())
    for safe in assets:
        df = datasets[safe].reset_index(drop=True)
        emb = embeddings.get(safe, np.zeros((0,)))
        for i in range(window, len(df)-horizon):
            window_df = df.iloc[i-window:i]
            feat = window_df[['o_pc','h_pc','l_pc','c_pc','v_pc']].values.astype(np.float32)
            # scale manually as above
            mean = scalers[safe]['mean'].values
            std = scalers[safe]['std'].values
            std = np.where(std==0,1e-8,std)
            feat_scaled = (feat - mean) / std
            emb_rep = np.tile(emb.reshape(1,-1),(window,1)) if emb.size>0 else np.zeros((window,0))
            balance_col = np.full((window,1),1.0,dtype=np.float32)
            obs = np.concatenate([feat_scaled, balance_col, emb_rep], axis=1)
            try:
                action, _ = model.predict(obs[np.newaxis,...], deterministic=True)
                action = int(action[0]) if isinstance(action,(list,tuple,np.ndarray)) else int(action)
            except Exception:
                continue
            if action == 0: continue
            entry_price = float(df['Close_raw'].iat[i-1])
            exit_price = float(df['Close_raw'].iat[i+horizon-1])
            pos = 1 if action==1 else -1
            pnl = (exit_price - entry_price)/entry_price * pos
            trades.append({"safe": safe, "entry_i": i, "horizon": horizon, "action":action, "pnl":pnl})
    return pd.DataFrame(trades)

# run quick sim
print("Running quick simulated backtest (may take a while)...")
df_trades = simulate_trades_on_history(datasets, model, assets=safe_list, window=WINDOW, horizon=10)
print("Simulated trades:", len(df_trades))
if not df_trades.empty:
    print("Total PnL:", df_trades['pnl'].sum(), "Win rate:", (df_trades['pnl']>0).mean())


Running quick simulated backtest (may take a while)...


In [ ]:
# Cell 12
if os.path.exists(LOG_FILE):
    df_log = pd.read_csv(LOG_FILE)
    display(df_log.tail(20))
else:
    print("No live logs yet at", LOG_FILE)
